In [137]:
import yfinance as yf
import optuna
import time
import numpy as np
import pandas as pd
from statsforecast import StatsForecast
from statsforecast.models import ARIMA
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import smape as smape_cv, mape as mape_cv, mae as mae_cv, mse as mse_cv, rmse as rmse_cv
from neuralforecast.losses.pytorch import SMAPE

import logging
import warnings

# Configuraciones para ignorar advertencias
warnings.simplefilter(action='ignore', category=FutureWarning)
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)
optuna.logging.set_verbosity(optuna.logging.WARNING)
logging.getLogger("lightning_fabric").setLevel(logging.WARNING)


In [138]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna
from neuralforecast import NeuralForecast
from neuralforecast.models import TiDE
from neuralforecast.losses.pytorch import SMAPE
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import smape as smape_cv, mape as mape_cv, mae as mae_cv, mse as mse_cv, rmse as rmse_cv
import logging
import warnings

# Configuraciones para ignorar advertencias
warnings.simplefilter(action='ignore', category=FutureWarning)
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)
optuna.logging.set_verbosity(optuna.logging.WARNING)
logging.getLogger("lightning_fabric").setLevel(logging.WARNING)

In [139]:
# Descargar datos
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'IBM', 'BTC-USD']
Y_df = yf.download(tickers, start='2000-01-01', interval='1mo')['Adj Close']

# Resetear el índice y transformar el DataFrame
Y_df = Y_df.reset_index().melt(id_vars='Date', var_name='unique_id', value_name='y')
Y_df = Y_df.rename(columns={'Date': 'ds'})
Y_df = Y_df[['unique_id', 'ds', 'y']]
Y_df = Y_df.dropna(subset=['y']).reset_index(drop=True)


[*********************100%%**********************]  8 of 8 completed


In [140]:
horizon = 1

In [170]:
def objective(trial, df_serie, horizon):
    try:
        # Definición de los parámetros a optimizar
        p = trial.suggest_int('p', 0, 5)
        d = trial.suggest_int('d', 0, 2)
        q = trial.suggest_int('q', 0, 5)
        P = trial.suggest_int('P', 0, 2)
        D = trial.suggest_int('D', 0, 1)
        Q = trial.suggest_int('Q', 0, 2)

        # Imprimir los parámetros seleccionados
        print(f"Parámetros seleccionados para el trial: p={p}, d={d}, q={q}, P={P}, D={D}, Q={Q}")

        # Crear el modelo ARIMA
        model = ARIMA(
            order=(p, d, q),
            seasonal_order=(P, D, Q, 12) 
        )

        # Pasar el modelo como una lista
        sf = StatsForecast(
            models=[model],  # Asegúrate de que model está en una lista
            freq='MS',  
            n_jobs=-1 
        )

        # Validación cruzada
        cv_df = sf.cross_validation(df_serie, n_windows=5, step_size=horizon)
        cv_df = cv_df.reset_index()

        # Evaluar las predicciones
        evaluation_df = evaluate(cv_df.loc[:, cv_df.columns != 'cutoff'],
                                 metrics=[smape_cv, mape_cv, mae_cv, mse_cv, rmse_cv])

        # Extraer métricas
        smape = evaluation_df[evaluation_df['metric'] == 'smape']['ARIMA'].values[0]
        mape = evaluation_df[evaluation_df['metric'] == 'mape']['ARIMA'].values[0]
        mae = evaluation_df[evaluation_df['metric'] == 'mae']['ARIMA'].values[0]
        mse = evaluation_df[evaluation_df['metric'] == 'mse']['ARIMA'].values[0]
        rmse = evaluation_df[evaluation_df['metric'] == 'rmse']['ARIMA'].values[0]

        # Guardar métricas en los atributos del trial
        trial.set_user_attr('mape', mape)
        trial.set_user_attr('mae', mae)
        trial.set_user_attr('mse', mse)
        trial.set_user_attr('rmse', rmse)

        return smape
    except Exception as e:
        print(f"Error en el trial para el ticker: {df_serie['unique_id'].unique()[0]}: {str(e)}")
        raise optuna.exceptions.TrialPruned()



In [171]:
best_params_by_ticker = {}

# Iterar sobre cada serie temporal
for ticker in tickers:
    print(f"Optimizando para {ticker}")

    # Filtrar los datos por ticker
    df_ticker = Y_df[Y_df['unique_id'] == ticker].reset_index(drop=True)

    # Crear el estudio de Optuna para cada ticker
    study = optuna.create_study(direction='minimize')

    # Optimizar los hiperparámetros
    study.optimize(lambda trial: objective(trial, df_ticker, horizon), n_trials=1)

    # Si el estudio tiene éxito, se imprimen los mejores resultados
    if study.best_trial:
        best_trial = study.best_trial
        mape = best_trial.user_attrs['mape']
        mae = best_trial.user_attrs['mae']
        mse = best_trial.user_attrs['mse']
        rmse = best_trial.user_attrs['rmse']

        best_params_by_ticker[ticker] = study.best_params

        print("Mejores hiperparámetros:", study.best_params)
        print("Mejor SMAPE:", study.best_value)
        print(f'Best MAPE: {mape}')
        print(f'Best MAE: {mae}')
        print(f'Best MSE: {mse}')
        print(f'Best RMSE: {rmse}')
    else:
        print("No hay trials completados exitosamente.")

Optimizando para AAPL
Parámetros seleccionados para el trial: p=2, d=2, q=1, P=2, D=0, Q=0
Error en el trial para el ticker: AAPL: can only concatenate str (not "int") to str


ValueError: No trials are completed yet.